# Práctica 9 Parte 3: Desarrollando un modelo de lenguaje para generar texto

Un modelo de lenguaje puede predecir la siguiente palabra de una secuencia basándose en palabras observadas anteriormente. Las redes neuronales son el método más utilizado para desarrollar este tipo de modelos porque pueden usar una representación donde palabras con significados similares tienen representaciones similares. 

En esta parte de la práctica vamos a ver cómo generar uno de esos modelos. 

Este notebook está basado en el libro Deep Learning for Natural Language Processing de Jason Brownlee. 

Es importante que tengas activado el uso de **GPU** en el notebook de colab (menú Edit -> Notebook Settings -> Hardware accelerator).

## La República de Platón

Nuestro modelo de lenguaje va a estar basado en la república de Platón. Este libro está estructurado en forma de una conversación que trata el tema del orden y la justicia dentro de una ciudad. El texto completo está disponible para el dominio público dentro del [proyecto Gutenberg](http://www.gutenberg.org/).

Este libro de Platón está disponible en varios formatos en el [proyecto Gutenberg](http://www.gutenberg.org/cache/epub/1497/pg1497.txt). La versión que nos interesa a nosotros es la versión ASCII del libro. Con la siguiente instrucción puedes descargar el libro donde se han eliminado la portada y la contraportada. 

In [ ]:
!wget https://raw.githubusercontent.com/ts1819/datasets/master/practica5/republic.txt -O republic.txt

--2022-06-12 00:56:19--  https://raw.githubusercontent.com/ts1819/datasets/master/practica5/republic.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 657826 (642K) [text/plain]
Saving to: ‘republic.txt’

republic.txt        100%[===================>] 642.41K  --.-KB/s    in 0.04s   

2022-06-12 00:56:19 (14.1 MB/s) - ‘republic.txt’ saved [657826/657826]



## Preparación de los datos

Vamos a preparar los datos para construir nuestro modelo. 

### Revisando el texto

Vamos a comenzar revisando parte del texto.

In [ ]:
!head -30 republic.txt

BOOK I.

I went down yesterday to the Piraeus with Glaucon the son of Ariston,
that I might offer up my prayers to the goddess (Bendis, the Thracian
Artemis.); and also because I wanted to see in what manner they would
celebrate the festival, which was a new thing. I was delighted with the
procession of the inhabitants; but that of the Thracians was equally,
if not more, beautiful. When we had finished our prayers and viewed the
spectacle, we turned in the direction of the city; and at that instant
Polemarchus the son of Cephalus chanced to catch sight of us from a
distance as we were starting on our way home, and told his servant to
run and bid us wait for him. The servant took hold of me by the cloak
behind, and said: Polemarchus desires you to wait.

I turned round, and asked him where his master was.

There he is, said the youth, coming after you, if you will only wait.

Certainly we will, said Glaucon; and in a few minutes Polemarchus
appeared, and with him Adeimantus, Glaucon's b

A partir de un rápido vistazo al fragmento de texto anterior podemos ver ciertas cuestiones que tendremos que procesar:
- Las cabeceras de los capítulos.
- Muchos signos de puntuación.
- Nombres extraños.
- Algunos monólogos muy largos. 

### Cargando el texto

El primer paso consiste en cargar el texto en memoria. Podemos desarrollar una pequeña función que se encargue de esto. 

In [ ]:
def load_doc(filename):
  # Abrimos el fichero en modo lectura
  file = open(filename,'r')
  # Leemos el texto completo
  text = file.read()
  # Cerramos el fichero
  file.close()
  return text

Usando dicha función podemos cargar nuestro fichero del siguiente modo.

In [ ]:
in_filename = 'republic.txt'
doc = load_doc(in_filename)

Ahora podemos mostrar parte de dicho texto.

In [ ]:
print(doc[:200])

BOOK I.

I went down yesterday to the Piraeus with Glaucon the son of Ariston,
that I might offer up my prayers to the goddess (Bendis, the Thracian
Artemis.); and also because I wanted to see in what


### Limpiando el texto

Ahora necesitamos transformar el texto en bruto a una secuencia de tokens (o palabras) que podamos usar para entrenar nuestro modelo. 

Vamos a aplicar las siguientes operaciones para limpiar nuestro texto:
- Reemplazar todas las ocurrencias de '-' con un espacio en blanco de manera que podamos partir mejor las palabras.
- Partir las palabras basándonos en espacios en blanco.
- Eliminar todos los símbolos de puntuación.
- Eliminar todas las palabras que no son alfabéticas. 
- Normalizar todas las palabras a minúsculas.

La mayoría de estas transformaciones tienen como objetivo reducir el tamaño del vocabulario. Un tamaño de vocabulario excesivamente grande es un problema cuando se intenta crear modelos de lenguaje. Vocabularios pequeños producen modelos más pequeños que se entrenan más rápidos.

Vamos a implementar todas las operaciones de limpieza en la siguiente función.

In [ ]:
import string
import re

def clean_doc(doc):
  # Reemplazar '--' con un espacio en blanco ' '
  doc = doc.replace('--',' ')
  # Partir palabras basándonos en espacios en blanco
  tokens = doc.split()
  # Vamos a escapar las palabras para poder filtrarlas por caracteres
  re_punc = re.compile('[%s]' % re.escape(string.punctuation))
  # Eliminamos los símbolos de puntuación
  tokens = [re_punc.sub('',w) for w in tokens]
  # Eliminamos elementos que nos son alfabéticos
  tokens = [word for word in tokens if word.isalpha()]
  # Convertimos a minúsculas
  tokens = [word.lower() for word in tokens]
  return tokens
  

Procedemos a limpiar nuestro documento y a continuación mostramos algunas estadísticas sobre nuestro vocabulario.

In [ ]:
tokens = clean_doc(doc)
print(tokens[:200])

['book', 'i', 'i', 'went', 'down', 'yesterday', 'to', 'the', 'piraeus', 'with', 'glaucon', 'the', 'son', 'of', 'ariston', 'that', 'i', 'might', 'offer', 'up', 'my', 'prayers', 'to', 'the', 'goddess', 'bendis', 'the', 'thracian', 'artemis', 'and', 'also', 'because', 'i', 'wanted', 'to', 'see', 'in', 'what', 'manner', 'they', 'would', 'celebrate', 'the', 'festival', 'which', 'was', 'a', 'new', 'thing', 'i', 'was', 'delighted', 'with', 'the', 'procession', 'of', 'the', 'inhabitants', 'but', 'that', 'of', 'the', 'thracians', 'was', 'equally', 'if', 'not', 'more', 'beautiful', 'when', 'we', 'had', 'finished', 'our', 'prayers', 'and', 'viewed', 'the', 'spectacle', 'we', 'turned', 'in', 'the', 'direction', 'of', 'the', 'city', 'and', 'at', 'that', 'instant', 'polemarchus', 'the', 'son', 'of', 'cephalus', 'chanced', 'to', 'catch', 'sight', 'of', 'us', 'from', 'a', 'distance', 'as', 'we', 'were', 'starting', 'on', 'our', 'way', 'home', 'and', 'told', 'his', 'servant', 'to', 'run', 'and', 'bid',

In [ ]:
print('Total Tokens: %d' % len(tokens))
print('Unique Tokens: %d' %len(set(tokens)))

Total Tokens: 118684
Unique Tokens: 7409


Es decir, nuestro modelo consta de una 7500 palabras. Este tamaño de vocabulario es pequeño y va a ser manejable.

### Guardando el texto limpio

Vamos a organizar la larga lista de tokens en secuencias de 50 palabras de entrada y 1 palabra de salida (esto servirá para luego entrenar nuestro modelo). 

Este proceso lo implementamos con la siguiente función.

In [ ]:
def organize_tokens(tokens,input_len=50,output_len=1):
  length = input_len + output_len
  sequences = list()
  for i in range(length,len(tokens)):
    # Elegimos la secuencia de tokens
    seq = tokens[i-length:i]
    # Convertimos la secuencia en una línea
    line = ' '.join(seq)
    # Almacenamos el resultado
    sequences.append(line)
  return sequences

Organizamos nuestros tokens. 

In [ ]:
lines = organize_tokens(tokens)

Ahora vamos a guardar las secuencias en un nuevo fichero para poder cargarlo en el futuro. Para ello nos definimos la siguiente función que guardará cada elemento de la secuencia en una línea del fichero. 

In [ ]:
def save_doc(lines,filename):
  data = '\n'.join(lines)
  file = open(filename,'w')
  file.write(data)
  file.close()

Podemos llamar a la función anterior para guardar nuestro fichero.

In [ ]:
out_filename = 'republic_sequences.txt'
save_doc(lines,out_filename)

Podemos ver parte de dicho fichero.

In [ ]:
!head -5 republic_sequences.txt

book i i went down yesterday to the piraeus with glaucon the son of ariston that i might offer up my prayers to the goddess bendis the thracian artemis and also because i wanted to see in what manner they would celebrate the festival which was a new thing i was
i i went down yesterday to the piraeus with glaucon the son of ariston that i might offer up my prayers to the goddess bendis the thracian artemis and also because i wanted to see in what manner they would celebrate the festival which was a new thing i was delighted
i went down yesterday to the piraeus with glaucon the son of ariston that i might offer up my prayers to the goddess bendis the thracian artemis and also because i wanted to see in what manner they would celebrate the festival which was a new thing i was delighted with
went down yesterday to the piraeus with glaucon the son of ariston that i might offer up my prayers to the goddess bendis the thracian artemis and also because i wanted to see in what manner they would

## Entrenando el modelo de lenguaje

Vamosa  entrenar ahora nuestro modelo a partir de los datos que hemos preparado. Dicho modelo tendrá ciertas características:
- Usará una representación para las palabras de manera que palabras diferentes con significados similares tendrán una representación similar.
- La representación será aprendida al mismo tiempo que se aprende el modelo.
- Aprenderá a predecir la probabilidad de la siguiente palabra a partir del contexto de las últimas 100 palabras.

En concreto para implementar este modelo vamos a usar una capa de Embedding para aprender la representación de las palabras, y una red neuronal recurrente con capas LSTM para predecir nuevas palabras basándonos en el contexto. 

### Cargando las secuencias

Podemos comenzar cargando las secuencias que hemos guardado anteriormente. En este caso este paso no sería necesario ya que el proceso de generación de las secuencias es bastante rápido, pero si estamos trabajando con un dataset más grande sí que puede ser conveniente. 


In [ ]:
in_filename = 'republic_sequences.txt'
doc = load_doc(in_filename)
lines = doc.split('\n')

### Codificando las secuencias

Las capas de Embedding esperan que las secuencias de entrada estén compuestas de vectores de enteros. Para ello vamos a identificar cada palabra de nuestro vocabulario con un entero único y codificarlo en una secuencia de entrada. En el futuro cuando vayamos a realizar las predicciones tendremos que realizar el proceso inverso.

Para llevar a cabo este proceso de tokenización vamos a usar la API de Keras del siguiente modo. 

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(lines)
sequences = tokenizer.texts_to_sequences(lines)

Ahora podemos acceder a los identificadores de cada palabra usando el atributo ``word_index`` del objeto ``Tokenizer`` que hemos creado. 

Además debemos determinar el tamaño de nuestro vocabulario para definir la capa de embedding. En concreto, a las palabras de nuestro vocabulario se les han asignado valores entre 1 y el número total de palabras de nuestro vocabulario. 

In [ ]:
vocab_size = len(tokenizer.word_index) + 1 

### Secuencias de entrada y salida

Una vez que tenemos codificadas nuestras secuencias tenemos que separarlas en elementos de entrada ($X$) y de salida ($y$). Después de realizar la separación debemos codificar cada palabra usando el método one-hot. Este proceso lo llevaremos a cabo mediante la función ``to_categorical()`` de Keras.
Finalmente necesitamos especificar cómo de largas serán las secuencias de entrada. 

In [ ]:
from tensorflow.keras.utils import to_categorical
from numpy import array 

sequences = array(sequences)
X,y=sequences[:,:-1], sequences[:,-1]
y = to_categorical(y,num_classes=vocab_size)
seq_length = X.shape[1]

### Entrenando el modelo

Ahora podemos definir nuestro modelo que constará de una capa de Embedding, seguida de dos capas LSTM y terminando con una red completamente conectada. 

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding
from tensorflow.keras.optimizers import Adam

def define_model(vocab_size,seq_length):
  model = Sequential()
  model.add(Embedding(vocab_size,50,input_length=seq_length))
  model.add(LSTM(100,return_sequences=True))
  model.add(LSTM(100))
  model.add(Dense(100,activation='relu'))
  model.add(Dense(vocab_size,activation='softmax'))
  model.compile(loss='categorical_crossentropy',optimizer=Adam(),metrics=['accuracy'])
  return model

Pasamos a entrenar nuestro modelo. Como este proceso es bastante costoso (incluso usando GPUs) en la siguiente sección se proporcionan los ficheros necesarios para usar el modelo. 

In [ ]:
model = define_model(vocab_size,seq_length)
model.fit(X,y,batch_size=128,epochs=100)

Epoch 1/100
927/927 [==============================] - 21s 15ms/step - loss: 6.1479 - accuracy: 0.0726
Epoch 2/100
927/927 [==============================] - 14s 15ms/step - loss: 5.6739 - accuracy: 0.1072
Epoch 3/100
927/927 [==============================] - 14s 15ms/step - loss: 5.4375 - accuracy: 0.1288
Epoch 4/100
230/927 [======>.......................] - ETA: 10s - loss: 5.3029 - accuracy: 0.1357

KeyboardInterrupt: ignored

Una vez entrenado podemos guardar los pesos del modelo y el tokenizador. 

In [ ]:
model.save_weights('./model.h5', overwrite=True)

In [ ]:
from pickle import dump
dump(tokenizer, open('tokenizer.pkl','wb'))

## Usando el modelo

Como has podido ver en el paso anterior, el proceso de entrenar este tipo de modelos es muy costoso, por lo que puedes descargar los ficheros necesarios para usar el modelo desde el siguiente enlace. 

In [ ]:
!wget https://raw.githubusercontent.com/ts1819/datasets/master/practica5/tpu_model.h5 -O model.h5
!wget https://raw.githubusercontent.com/ts1819/datasets/master/practica5/tokenizer.pkl -O tokenizer.pkl

--2022-06-11 23:06:56--  https://raw.githubusercontent.com/ts1819/datasets/master/practica5/tpu_model.h5
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5101248 (4.9M) [application/octet-stream]
Saving to: ‘model.h5’

model.h5            100%[===================>]   4.86M  --.-KB/s    in 0.08s   

2022-06-11 23:06:57 (63.1 MB/s) - ‘model.h5’ saved [5101248/5101248]

--2022-06-11 23:06:57--  https://raw.githubusercontent.com/ts1819/datasets/master/practica5/tokenizer.pkl
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 353379 (345K)

### Cargando los datos

Comenzamos cargando nuestros datos al igual que antes. 

In [ ]:
in_filename = 'republic_sequences.txt'
doc = load_doc(in_filename)
lines = doc.split('\n')

Necesitamos este texto para elegir una secuencia de inicio que será la entrada para nuestro modelo. 

In [ ]:
seq_length = len(lines[0].split())-1

### Cargando el modelo

Vamos a cargar el modelo y a fijar los pesos. Notar que para este paso ya no necesitamos el uso de TPU, y que el modelo podría ser usado en cualquier ordenador.

In [ ]:
model = define_model(vocab_size,seq_length)
model.load_weights('./model.h5')

También necesitamos cargar el tokenizador.

In [ ]:
from pickle import load
tokenizer = load(open('tokenizer.pkl','rb'))

### Generando texto

El primer paso para generar el texto consiste en preparar una entrada, para lo cual elegiremos una línea aleatoria del texto. 

In [ ]:
from random import randint
seed_text = lines[randint(0,len(lines))]
print(seed_text + '\n')

done but is war an art so easily acquired that a man may be a warrior who is also a husbandman or shoemaker or other artisan although no one in the world would be a good dice or draught player who merely took up the game as a recreation and had



A continuación podemos generar nuevas palabras una por una. Primero, el texto debe codificarse usando el tokenizer que hemos cargado anteriormente. Ahora el modelo puede predecir nuevas palabras usando el método ``predict_classes()`` que devuelve el índice de la palabra con probabilidad más alta. 

Esta palabra se añade a nuestro texto inicial y se repite el proceso. Notar que esta secuencia va a ir creciendo por lo que tendremos que truncarla, para lo que utilizamos la función ``pad_sequences()`` de Keras. Todo este proceso se puede implementar con la siguiente función. 

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

def generate_seq(model,tokenizer,seq_length,seed_text,n_words):
  result = list()
  in_text = seed_text
  for _ in range(n_words):
    encoded = tokenizer.texts_to_sequences([in_text])[0]
    encoded = pad_sequences([encoded],maxlen=seq_length,truncating='pre')
    yhat = model.predict(encoded,verbose=0)
    yhat=np.argmax(yhat,axis=1)
    out_word = ''
    for word,index in tokenizer.word_index.items():
      if index == yhat:
        out_word = word
        break
    in_text += ' ' + out_word
    result.append(out_word)
  return ' '.join(result)

Ahora podemos generar una nueva secuencia usando el siguiente código. Cada vez que lo ejecutemos obtendremos un resultado distinto.

In [ ]:
seed_text = lines[randint(0,len(lines))]
print(seed_text + '\n')
generated = generate_seq(model,tokenizer,seq_length,seed_text,50)
print(generated)

in introducing them to dialectic certainly there is a danger lest they should taste the dear delight too early for youngsters as you may have observed when they first get the taste in their mouths argue for amusement and are always contradicting and refuting others in imitation of those who refute

them and he will be preserved in his head and says that the other rascalities and the other creative and constructive city are the just man and the other allurements of justice and therefore i said the question is the better and exhibits to them his life in finding money


## Ejercicio

Elige tu propio libro del proyecto Gutenberg (es posible usar libros en [español](https://www.gutenberg.org/browse/languages/es)) y crea tu propio modelo de lenguaje. 

In [ ]:
don_comedia = 'Divina comedia.txt'
doc2 = load_doc(don_comedia)

In [ ]:
tokens2 = clean_doc(doc2)
lines2 = organize_tokens(tokens2)

In [ ]:
salida_comedia = 'Comedia_graciosa.txt'
save_doc(lines2,salida_comedia)

In [ ]:
entrada_comedia = 'Comedia_graciosa.txt'
doc2 = load_doc(entrada_comedia)
lines2 = doc.split('\n')

In [ ]:
tokenizer2 = Tokenizer()
tokenizer2.fit_on_texts(lines2)
sequences2 = tokenizer2.texts_to_sequences(lines2)
comedia_size = len(tokenizer2.word_index) + 1 

In [ ]:
sequences = array(sequences)
X,y=sequences[:,:-1], sequences[:,-1]
y = to_categorical(y,num_classes=comedia_size)
comedia_length = X.shape[1]

In [ ]:
def define_model(comedia_size,comedia_length):
  model = Sequential()
  model.add(Embedding(vocab_size,50,input_length=seq_length))
  model.add(LSTM(100,return_sequences=True))
  model.add(LSTM(100))
  model.add(Dense(100,activation='relu'))
  model.add(Dense(vocab_size,activation='softmax'))
  model.compile(loss='categorical_crossentropy',optimizer=Adam(),metrics=['accuracy'])
  return model

In [ ]:
modelComedia = define_model(comedia_size,comedia_length)
modelComedia.fit(X,y,batch_size=128,epochs=100)

In [39]:
modelComedia.save_weights('./modelComedia.h5', overwrite=True)

In [40]:
tokenizer = load(open('tokenizer.pkl','rb'))
seed_text = lines2[randint(0,len(lines2))]
print(seed_text + '\n')

kindly answer for the edification of the company and of myself glaucon and the rest of the company joined in my request and thrasymachus as any one might see was in reality eager to speak for he thought that he had an excellent answer and would distinguish himself but at first



In [41]:
seed_text = lines2[randint(0,len(lines2))]
print(seed_text + '\n')
generated = generate_seq(modelComedia,tokenizer,comedia_length,seed_text,50)
print(generated)

and love are fled away there was a good time once but now that is gone and life is no longer life some complain of the slights which are put upon them by relations and they will tell you sadly of how many evils their old age is the cause but

in the tyrannical state socrates and the spirits arrived in the city of which the senses are imposed upon by distance and by painting or fellowsailors within the end of the visible and assuredly with husbandry with the other particular things of which we may write and speak of false


Recuerda guardar este notebook en tu repositorio usando la opción "Save in GitHub" del menú File.